In [1]:
from attitude import Orientation
from attitude.display import plot_interactive, init_notebook_mode
import numpy as np

In [7]:
def setPCAOrientation(input_dips):
    #takes in list of dips from Daven's code and changes orientation to be 0N, 90E, 180S, 270W
    #Rotate 90deg CW and flip on y axis
    input_adj = []
    transient_dip = 0
    for i in range(len(input_dips)):
        if input_dips[i] < 0:
            input_dips[i] = input_dips[i] + 360
        if input_dips[i] < 270:
            transient_dip = input_dips[i] + 90
        else:
            transient_dip = input_dips[i] + 90 - 360
        if transient_dip < 90:
            input_adj.append(270+(90-transient_dip)) 
        elif transient_dip < 180 and transient_dip > 90:
            input_adj.append(180+(180-transient_dip))
        elif transient_dip < 270 and transient_dip > 180:
            input_adj.append(90+(270-transient_dip))
        elif transient_dip < 360 and transient_dip > 270:
            input_adj.append(360-transient_dip)
    return input_adj

In [8]:
def average_dip_direction(dip_directions,dip_mags):
    #8/15 note: this function isn't used right now.
    import math
    import statistics as stat
    dip_directions_x = []
    dip_directions_y = []
    dip_directions_rad = []
    for i in range(len(dip_directions)):
        dip_directions_rad.append(math.radians(dip_directions[i]))
        dip_directions_x.append(dip_mags[i]*math.cos(dip_directions_rad[i])) #x component of dip direction unit vectors
        dip_directions_y.append(dip_mags[i]*math.sin(dip_directions_rad[i])) #y component of dip directions unit vector
    avg_x_vec = stat.mean(dip_directions_x)
    avg_y_vec = stat.mean(dip_directions_y)
    avg_dip_direction = math.degrees(math.atan2(avg_y_vec,avg_x_vec))
    #avg_dip_direction = setPCAOrientation([avg_dip_direction])
    avg_dip_mag       = math.sqrt(avg_x_vec**2 + avg_y_vec**2)
    return avg_dip_direction, avg_dip_mag

In [9]:
def variance_angle(deg):
    """
    calculates circular variance
    deg: angles in degrees 
    """
    deg = np.deg2rad(deg)
    deg = deg[~np.isnan(deg)]

    S = np.array(deg)
    C = np.array(deg)

    length = C.size

    S = np.sum(np.sin(S))
    C = np.sum(np.cos(C))
    R = np.sqrt(S**2 + C**2)
    R_avg = R/length
    V = 1- R_avg

    return V

In [10]:
def gen_data_PCA(data,errors,num_point_sets):
    #Read in XYZ and XYE points and generate different sets of xyz points for use in PCA model
    import numpy as np
    import scipy.linalg
    import random
    import statistics
    
    data_new = 0
    dips       = []
    dip_angles = []
    x_err      = []
    y_err      = []
    z_err      = []
    x          = []
    init_x     = []

    data_new = np.empty((data.shape[0]*num_point_sets,data.shape[-1]))
    print(data.shape)
    for q in range(num_point_sets):
        for i in range(len(data)):
            for j in range(data.shape[1]):
                data_new[i+q*len(data)][j] = data[i][j] + random.gauss(0,errors[i][j])#+random.gauss(0,0.364*errors[i][j]) #.364 assumes that the errors denote the 3 sigma confidence and scale the standard deviation accordingly (Z-score of 2.75)
                if j == 0:
                    x_err.append(errors[i][j])
                    x.append(data_new[i][j]) #just book keeping, ignore
                    init_x.append(data[i][j]) #just book keeping
                elif j == 1:
                    y_err.append(errors[i][j])
                elif j == 2:
                    z_err.append(errors[i][j])
    return data_new, statistics.mean(x_err), statistics.mean(y_err), statistics.mean(z_err),x,init_x

In [14]:
#VERSION WHERE ONLY ONE PCA RUN IS PERFORMED ON MONTE CARLO POINTS

#Read in xyz and xye data and save out txt file containing n sets of possible xyz arrangements using xye
import numpy as np
import scipy.linalg
import os
import random
import matplotlib.pyplot as plt
import statistics
import math
from scipy.stats import circmean
from openpyxl import load_workbook

#################IMPORT DATA##################
#Read xyz data
roi_start_tracker = []
file = "C:/Users/ntste/Documents/Mars/VRR/Dips/Monte Carlo Dips/Data by Sol/1925/2040_MR_roi_BP_actuallyMR_xyz.txt"
xyz = np.genfromtxt(file, comments=";",dtype=float,usecols=np.arange(0,6))
xyz = xyz.tolist()
for i in range(len(xyz)):
    if xyz[i][0] == 1.0:
        roi_start_tracker.append(i)
    del xyz[i][0:3]
data = np.asarray(xyz)

#Read xyz errors
file = "C:/Users/ntste/Documents/Mars/VRR/Dips/Monte Carlo Dips/Data by Sol/1925/2040_MR_roi_BP_actuallyMR_xye.txt"
xyz = np.genfromtxt(file, comments=";",dtype=float,usecols=np.arange(0,6))
xyz = xyz.tolist()
for i in range(len(xyz)):
    del xyz[i][0:3]
errors = np.asarray(xyz)
###############################################

##################INITIALIZE###################
num_point_sets = 1000 #Number of monte carlo points to generate for each input point 

avg_x_err             = [] #Average x error of each pixel in trace
avg_y_err             = [] #Average y error of each pixel in trace
avg_z_err             = [] #Average z error of each pixel in trace
data_new              = [] #set of Monte Carlo generated xyz points for input to Daven's method
dip_directions        = [] #Dip directions output by Daven's method
dip_directions_MSL    = [] #Dip directions corrected to Mars frame
dip_angles            = [] #Dip angles output by Daven's method
rake                  = [] #Rake output by Daven's method
mine                  = [] #Mine output by Daven's method
maxe                  = [] #Maxe output by Daven's method
dx                    = [] #Change in x along trace (m)
dy                    = [] #Change in y along trace (m)
dz                    = [] #Change in z along trace (m)
eigen1                = [] #Eigenvector 1 output from Daven's method
eigen2                = [] #e2
eigen3                = [] #e3
num_pts               = [] #Number of points per bed trace (excludes Monte Carlo population)
trace_length          = [] #Trace length (m)
###############################################

for n in range(len(roi_start_tracker)): #Step through all ROIs in the .txt files one by one
    print(n)
    
    ###############GENERATE DATA AND APPLY DAVEN'S METHOD###############
    if n < len(roi_start_tracker) -1:
        end_index = roi_start_tracker[n+1]
    else:
        end_index = len(data)
    num_pts.append(end_index - roi_start_tracker[n]) #Number of points in each bed fit (before Monte Carlo population)
    dip_directions    = []
    dip_directions_r  = []
    data2             = []
    data_new, x_err, y_err, z_err,x,init_x = gen_data_PCA(data[:][roi_start_tracker[n]:end_index],errors[:][roi_start_tracker[n]:end_index],num_point_sets)
    data2             = np.asarray(data_new)
    
    orientations = Orientation(data2[:][0:len(data_new)])
    dip_info = orientations.dip_direction(0)
    dip_directions.append(dip_info[0])
    dip_directions_r.append(math.radians(dip_info[0])) #Vestigial from a previous version that computed circular variance of Monte Carlo solutions
    dip_angles.append(dip_info[1])
    ###################################################################
    
    for i in range(len(dip_directions)):
        if dip_directions[i] > 360:
            dip_directions[i] -= 360    
    avg_x_err.append(x_err)
    avg_y_err.append(y_err)
    avg_z_err.append(z_err)
    angular_errors = orientations.angular_errors()
    rake.append(orientations.rake())
    mine.append(angular_errors[0])
    maxe.append(angular_errors[1])
    eigen1.append(orientations.eigenvalues[0])
    eigen2.append(orientations.eigenvalues[1])
    eigen3.append(orientations.eigenvalues[2])  
    
    data_n = data[:][roi_start_tracker[n]:end_index]
    x = []
    y = []
    z = []
    for l in range(data_n.shape[0]):
        for p in range(data_n.shape[1]):
            if p == 0:
                x.append(data_n[l][p])
            elif p == 1:
                y.append(data_n[l][p])
            elif p == 2:
                z.append(data_n[l][p])    
    dx.append(max(x)-min(x))
    dy.append(max(y)-min(y))
    dz.append(max(z)-min(z))
    trace_length.append(math.sqrt(dx[n]**2 + dy[n]**2 + dz[n]**2)) #total trace length
    dip_directions_MSL.append(setPCAOrientation(dip_directions))
    
    print(dip_angles[n], dip_directions_MSL[n])
    print(trace_length[n])
    print(eigen1[n])


0
(674, 3)
8.131661193283103 [155.45404703415574]
0.2899807062547579
0.006892248945563927
1
(168, 3)
12.51435995525508 [158.5433317658184]
0.08576974991218958
0.0005427396992676376
2
(137, 3)
6.424229972195093 [104.59238467610498]
0.07013765037410823
0.00047367244194991434
3
(156, 3)
6.2473058197448665 [109.03119951025576]
0.07356384981768964
0.0005267971718664898
4
(168, 3)
4.759597926085178 [54.75737611400564]
0.061193545411253644
0.00037774924096940023
5
(149, 3)
3.1635602455509213 [120.80394094968634]
0.052756610960151565
0.00024602983329425845
6
(185, 3)
5.083735979441079 [151.85557932566292]
0.06701619207326327
0.00034241812279630355
7
(194, 3)
5.511253982493827 [147.65829172136483]
0.06649037524334014
0.0003550780262364449
8
(349, 3)
14.021648913610505 [147.25581911067957]
0.13429478768739725
0.0012957983012282595
9
(380, 3)
7.414917213038653 [157.92340851831855]
0.14914057127421554
0.0018947691475030524
10
(130, 3)
10.193946305108312 [174.1246511900521]
0.08135619214294718
0.00

In [ ]:
#GENERATE XLSX FILE WITH RESULTS FROM PCA WITH MONTE CARLO SINGLE RUN
from openpyxl.workbook import Workbook
#MAKE NEW
headers       = ['Sol','ID','n','L','Dip','Dip Direction','Eigenvalue 1', 'Eigenvalue 2', 'Eigenvalue 3','Rake','Min_e','Max_e','Avg x err','Avg y err','Avg z err']
path = 'C:/Users/ntste/Documents/Mars/VRR/Dips/Monte Carlo Dips/all_PCA_dips_montecarlo_BP_singlerun_one_std_w_errors.xlsx'
wb = Workbook()
page = wb.active
page.append(headers) # write the headers to the first line


# Data to write
input_data = []
for i in range(len(rake)):
    input_data.append([1802,5266,num_pts[i],trace_length[i], dip_angles[i], dip_directions_MSL[i][0], eigen1[i], eigen2[i], eigen3[i], rake[i], mine[i], maxe[i], avg_x_err[i], avg_y_err[i], avg_z_err[i]])

for info in input_data:
    page.append(info)
wb.save(filename = path)

In [ ]:
#APPEND NEW DATA TO XLSX FILE WITH RESULTS FROM PCA WITH MONTE CARLO SINGLE RUN
from openpyxl import load_workbook

path = 'C:/Users/ntste/Documents/Mars/VRR/Dips/Monte Carlo Dips/all_PCA_dips_montecarlo_BP_singlerun_one_std_w_errors.xlsx'
wb = load_workbook(path)
page = wb.active

# Data to write:
input_data = []
for i in range(len(rake)):
    input_data.append([2259,3092,num_pts[i],trace_length[i], dip_angles[i], dip_directions_MSL[i][0],eigen1[i], eigen2[i], eigen3[i], rake[i], mine[i], maxe[i], avg_x_err[i], avg_y_err[i], avg_z_err[i]])

for info in input_data:
    page.append(info)
wb.save(filename = path)